In [32]:
import numpy as np
import matplotlib.pyplot as plt
import math
import control as co
from control.matlab import *
from scipy import signal
from IPython.display import Image
from IPython.core.display import HTML 
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual


## Control mediante retroalimentación de estados
Considere el sistema definido por las ecuaciones de estado
\begin{eqnarray}
\dot{\textbf{x}}&=&\textbf{A}\cdot \textbf{x}+\textbf{B} \cdot \textbf{u} \\
\label{eedoder}
y&=&\textbf{C} \cdot \textbf{x} + \textbf{D} \cdot \textbf{u}
\end{eqnarray} 
	
Si se aplica la transformada de Laplace a la ecuación anterior se tiene:
\begin{equation}
sX(s)=AX(s)+BU(s)
\end{equation}
	
Despejando $X(s)$, se tiene:
\begin{equation}
sX(s)-AX(s)=BU(s)
\end{equation}
	
Y factorizando:
\begin{equation}
[sI-A]X(s)=BU(s)
\end{equation}
	
y despejando $X(s)$ se tiene:
\begin{equation}
	X(s)=[sI-A]^{-1}BU(s)
\end{equation}

De la ecuación anterior se tiene que la ecuación característica es:
\begin{equation}
|s\textbf{I}-\textbf{A}|=0
\end{equation}
donde los eigenvalores del determinante, son los polos del sistema.
 

Si las variables de estado $\textbf{x}$ se retroalimentan.

<img src="img/controlt/retross2.png" width="400px"> 

La ecuación característica será con $u=-K\textbf{x}$ y $r(t)=0$, es:
\begin{equation}
|s\textbf{I}-\textbf{A}+\textbf{BK}|=0
\end{equation}
 
### Controlabilidad
Para aplicar el control por retroalimentación de estados el sistema debe ser controlable, esto se cumple sí y solo sí, la matriz 
\begin{equation}
\textbf{S}=[B\ \; AB \; A^2B \; \ldots \; A^{n-1}B]
\end{equation}
es de rango n. 
	
### Definición: Rango de una matriz
Es el orden del mayor determinante distinto de cero que se puede encontrar en el interior de una matriz determinada.
	
**Ejercicio**
Determine si los siguientes sistemas son controlables.
\begin{eqnarray}
\begin{bmatrix}
\dot{x1} \\ \dot{x2}
\end{bmatrix}
=
\begin{bmatrix}
1 & 1 \\ 0 & -1
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 
\end{bmatrix}
+
\begin{bmatrix}
1  \\ 0
\end{bmatrix}
u
\end{eqnarray}
	
\begin{eqnarray}
\begin{bmatrix}
\dot{x1} \\ \dot{x2}
\end{bmatrix}
=
\begin{bmatrix}
1 & 1 \\ 2 & -1
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 
\end{bmatrix}
+
\begin{bmatrix}
0  \\ 1
\end{bmatrix}
u
\end{eqnarray}
 
#### Teoremas de retroalimentación de estados
* Una condición necesaria y suficiente para la colocación arbitraria de los polos es que el sistema sea de estado completamente controlable.
* Si el par [**A,B**] es completamente controlable, existe una matriz $\textbf{K}$ que puede dar un conjunto arbitrario de valores característicos de $(\textbf{A}-\textbf{B}\textbf{K})$; las $\textit{n}$ raíces de la ecuación característica:

\begin{equation}
|s\textbf{I}-\textsf{A}+\textbf{B}\textbf{K}|
\end{equation}

**Ejercicio**

Diseñe un controlador por retroalimentación de estados del siguiente sistema 
\begin{equation}
G(s)=\frac{2500}{s(s+25)}
\end{equation}
para los polos se ubiquen en $s_1=-50$ y $s_2=-50$. 

**Hint:** 
La función de transferencia se puede representar como $G(S)=\frac{Y(s)}{R(s)}$. A continuación convierta la ecuación al dominio del tiempo y obtenga el diagrama. 
<img src="img/controlt/diagrama_ssv2.png" width="600px">
De este diagrama pueden obtenerse las ecuaciones de estado. Las variables de estado se ubican en las salidas de los integradores, como puede observarse. Las ecuaciones de estado son las siguientes.

\begin{eqnarray}
\begin{bmatrix}
\dot{x1} \\ \dot{x2}
\end{bmatrix}
=
\begin{bmatrix}
0 & 1 \\ 0 & -25
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 
\end{bmatrix}
+
\begin{bmatrix}
0  \\ 2500
\end{bmatrix}
r(t)
\end{eqnarray}

In [17]:
A = [[0, 1], [0, -25]]
B = [[0], [2500]]
p = [-50, -49.9999]
co.place(A, B, p)

matrix([[0.999998  , 0.02999996]])

## Retroalimentación de estados con control integral
Para eliminar el error en estado estable se agrega un integrador y por otra parte se agrega la señal de referencia ($r(t) \neq 0$). De tal manera que aumenta en uno las variables de estado.

<img src="img/controlt/retroint.png" width="400px"> 
    
\begin{eqnarray}
\begin{bmatrix}
\dot{\textbf{x}(t)}\\
\dot{x_{n+1}}\\
\end{bmatrix}
=\begin{bmatrix}
\textbf{A} & 0\\
-\textbf{C} & 0\\
\end{bmatrix}
\begin{bmatrix}
\textbf{x}(t)\\
x_{n+1}\\
\end{bmatrix}
+
\begin{bmatrix}
\textbf{B}\\
\textbf{-D}\\
\end{bmatrix}
u(t)
+
\begin{bmatrix}
\textbf{0}\\
1\\
\end{bmatrix}
r(t)
\end{eqnarray}

ya que $u=-\textbf{Kx}-K_{n+1}x_{n+1}$

\begin{eqnarray*}
	\begin{bmatrix}
		\dot{\textbf{x}(t)}\\
		\dot{x_{n+1}}\\
	\end{bmatrix}
	=
	\begin{bmatrix}
		\textbf{A} & 0\\
		-\textbf{C} & 0\\
	\end{bmatrix}
	\begin{bmatrix}
		\textbf{x}(t)\\
		x_{n+1}\\
	\end{bmatrix}
	-
	\begin{bmatrix}
		\textbf{B}\\
		\textbf{-D}\\
	\end{bmatrix}
	\begin{bmatrix}
		\textbf{K} & k_{n+1}\\
	\end{bmatrix}
	\begin{bmatrix}
		\textbf{x}(t)\\
		x_{n+1}\\
	\end{bmatrix}
	+
	\begin{bmatrix}
		\textbf{0}\\
		1\\
	\end{bmatrix}
	r(t)
\end{eqnarray*}
	
La ecuación de estado puede expresarse como:
\begin{equation}
\dot{{\hat{\textbf{x}}}}=\hat{\textbf{A}}\hat{\textbf{x}}-\hat{\textbf{B}} \hat{\textbf{K}} \hat{\textbf{x}}+
\begin{bmatrix}
\textbf{0}\\
1\\
\end{bmatrix}
r(t)
\end{equation}
 
Y la ecuación característica para el sistema es:
\begin{equation}
|s{\textbf{I}}-{\hat{\textbf{A}}}+\hat{\textbf{B}}\hat{\textbf{K}}|=0
\end{equation}
donde:
\begin{eqnarray}
\hat{\textbf{A}}=
\begin{bmatrix}
\textbf{A} & 0\\
-\textbf{C} & 0\\
\end{bmatrix}, 
\hat{\textbf{B}}=\begin{bmatrix}
\textbf{B}\\
\textbf{-D}\\
\end{bmatrix},
\hat{\textbf{K}}=
\begin{matrix}
[k_1 & k_2 & k_3 \:  ... \: & k_{n+1}]
\end{matrix}
\end{eqnarray}
 


## Observabilidad
Un sistema es completamente observable si cada variable de estado del sistema afecta alguna de las salidas.
	
#### Teorema
Para que el sistema descrito por las ecuaciones: 
\begin{eqnarray}
\dot{\textbf{x}}&=&\textbf{A}\cdot \textbf{x}+\textbf{B} \cdot \textbf{u} \\
y&=&\textbf{C} \cdot \textbf{x} + \textbf{D} \cdot \textbf{u}
\end{eqnarray}
sea completamente observable es necesario y suficiente que la matriz:
\begin{eqnarray}
V=
\begin{bmatrix}
C & CA & CA^2 & \: ... \:CA^{n-1}
\end{bmatrix}
^T
\end{eqnarray}
sea de rango n.




**Ejercicio**

Determine si los siguientes sistemas son observables.
\begin{eqnarray}
A=
\begin{bmatrix}
-2 & 0 \\
0 & -1 \\ 
\end{bmatrix} 
B=
\begin{bmatrix}
3  \\
1  \\
\end{bmatrix} \:
C=
\begin{bmatrix}
1 & 0 
\end{bmatrix} 
\end{eqnarray}
	
\begin{eqnarray}
A=
	\begin{bmatrix}
	0 & 1 \\
	-2 & -3 \\ 
	\end{bmatrix} 
	B=
	\begin{bmatrix}
	1  \\
	1  \\
	\end{bmatrix} \:
	C=
	\begin{bmatrix}
	1 & 2 
	\end{bmatrix} 
	\end{eqnarray}
	
Para el sistema de levitación magnética calcule la observabilidad, si la salida es la posición, la velocidad o la corriente.
 

In [49]:
A = [[-2, 0], [0, -1]]
C = [3, 0]

In [50]:
C

[3, 0]

In [51]:
co.obsv(A,C)

matrix([[ 3.,  0.],
        [-6.,  0.]])

In [53]:
A = [[0, 1], [-2, -3]]
C = [[1, 2]]
O2=co.obsv(A,C)

In [55]:
np.ptp(O2)

7.0

## Observador de orden completo
Para un sistema representado con variables de estado, se puede obtener un observador de orden completo de la siguiente forma:
\begin{equation}
\dot {\tilde {\textbf{x}}} = \textbf{A} \tilde{ {\textbf{x}}} + \textbf{B}u+\textbf{K}_e(y- \textbf{C} \tilde { \textbf{x}})
\end{equation}

<img src="img/controlt/obscom.png" width="400px"> 

El problema de diseñar un observador de orden completo está en determinar la matriz de ganancias del observador $\textbf{K}_e$ de forma que la dinámica del error sea asintóticamente estable con una velocidad de respuesta suficiente. \\
	
Restando el sistema original del observador se tiene:
\begin{equation}
\dot{\textbf{x}} - \dot {\tilde {\textbf{x}}} = \textbf{A}({\textbf{x}} - {\tilde {\textbf{x}}}) -\textbf{K}_e \textbf{C} ({\textbf{x}} - {\tilde {\textbf{x}}})
\end{equation}
	
Los valores propios deseados de la ecuación característica se deben escoger de forma que el observador de estados responda de 2 a 5 veces más rápido que el sistema en lazo cerrado. La ecuación característica del observador es:
\begin{equation}
|s\textbf{I}-\textbf{A}+\textbf{K}_e \textbf{C}|=0
\end{equation}
 

Ejercicio
Considere el sistema representado mediante variables de estado, con:
\begin{eqnarray}
A=
\begin{bmatrix}
0 & 20.6 \\
1 & 0 \\ 
\end{bmatrix} 
\:
B=
\begin{bmatrix}
0 \\
1  \\
\end{bmatrix} \:
C=
\begin{bmatrix}
0 & 1 
\end{bmatrix} 
\end{eqnarray}
	
Determine la matriz de ganancias $\textbf{K}_e$ tal que los valores propios deseados de la matriz del observador sean $\gamma _1 =-5$ y $\gamma _2 =-5$. Considere que $u=-\textbf{K}\tilde{\textbf{x}}$.



In [58]:
A = [[0, 20.6], [1, 0]]
B = [[0], [1]]
C = [0, 1]
p = [-5, -4.99]
co.place(A, B, p)

matrix([[2.21116505, 9.99      ]])

## Observador de orden mínimo
Algunas de las variables se pueden estimar a partir de una señal que puede ser medida; en otras palabras a partir de la señal medida se observan las otras señales. Considerando el caso en el que solo una variable de estado es medible, para el sistema:

\begin{eqnarray}
\dot{\textbf{x}}&=&\textbf{A}\cdot \textbf{x}+\textbf{B} \cdot \textbf{u} \\
y&=&\textbf{C} \cdot \textbf{x} + \textbf{D} \cdot \textbf{u}
\end{eqnarray} 
 
La ecuación de estado puede reescribirse como:
\begin{eqnarray}
\dot{\textbf{x}}=
\begin{bmatrix}
\dot{x_a} \\
\dot{\textbf{x}_b}
\end{bmatrix}
=
\begin{bmatrix}
A_{aa} & \textbf{A}_{ab} \\
\textbf{A}_{ba} & \textbf{A}_{bb}
\end{bmatrix}
\begin{bmatrix}
x_a \\
\textbf{x}_b
\end{bmatrix}
+
\begin{bmatrix}
B_a\\
\textbf{B}_b
\end{bmatrix}
u
\end{eqnarray}
	
\begin{eqnarray}
y=
\begin{bmatrix}
1&0
\end{bmatrix}
\begin{bmatrix}
x_a \\
\textbf{x}_b
\end{bmatrix}
\end{eqnarray}
de donde:
\begin{eqnarray}
\dot{x_a}=A_{aa}x_a+\textbf{A}_{ab}\textbf{x}_b+B_au\\
\dot{\textbf{x}_b}=\textbf{A}_{ba}x_a+\textbf{A}_{bb}\textbf{x}_b+\textbf{B}_bu
\end{eqnarray}
 

 
\begin{eqnarray}
\dot{x_a}=A_{aa}x_a+\textbf{A}_{ab}\textbf{x}_b+B_au\\
\dot{\textbf{x}_b}=\textbf{A}_{ba}x_a+\textbf{A}_{bb}\textbf{x}_b+\textbf{B}_bu
\end{eqnarray}
Haciendo
\begin{eqnarray}
y^*=\dot{x_a}-A_{aa}x_a-B_au,\, \textbf{C}^*=\textbf{A}_{ab},\,\textbf{B}^*u^*= \textbf{A}_{ba}x_a+\textbf{B}_bu
\end{eqnarray}
Se tiene
\begin{eqnarray}
\dot{\textbf{x}_b}&=&\textbf{A}_{bb}\textbf{x}_b+\textbf{B}^*u^*\\
y^*&=&\textbf{C}^*\textbf{x}_b 
\end{eqnarray}
 

Sustituyendo con los cambios de variables realizados, se tiene: 
\begin{eqnarray}
\dot{\textbf{x}_b}&=&\textbf{A}_{bb}\textbf{x}_b+\textbf{A}_{ba}x_a+\textbf{B}_bu\\
\dot{\textbf{x}_b}&=&\textbf{A}_{bb}\textbf{x}_b+\textbf{B}^*u^*\\
\dot{x_a}-A_{aa}x_a-B_au&=&\textbf{A}_{ab}\textbf{x}_b\\% y^*&=&\textbf{C}^*\textbf{x}_b \\
\end{eqnarray}
	
Ahora considere la ecuación del observador de orden completo
\begin{equation}
\dot {\tilde {\textbf{x}}} = \textbf{A} \tilde{ {\textbf{x}}} + \textbf{B}u+\textbf{K}_e(y- \textbf{C} \tilde { \textbf{x}}) = (\textbf{A}-\textbf{K}_e \textbf{C} )\tilde{ {\textbf{x}}} + \textbf{B}u+ \textbf{K}_e y
\end{equation}
Haciendo cambios de variables 
$\tilde{\textbf{x}}=\tilde{\textbf{x}_b}$, $\textbf{A}=\textbf{A}_{bb}$, $\textbf{C}=\textbf{A}_{ab}$, $\textbf{B}\textbf{u}= \textbf{A}_{ba}x_a+\textbf{B}_bu$ y $y=\dot{x_a}-A_{aa}x_a-B_au$.
 

Después de sustituir queda la ecuación
\begin{equation}
\tilde {\dot{\textbf{x}_b}}=(\textbf{A}_{bb}-\textbf{K}_e\textbf{A}_{ab})\tilde {\textbf{x}_b}+ \textbf{A}_{ba}x_a+\textbf{B}_bu+\textbf{K}_e(\dot{x_a}-A_{aa}x_a-B_au)
\end{equation}
	
Al estimar $\tilde{\textbf{x}_b}$ se requiere la derivada de $x_a$, pero si ésta señal es ruidosa, la derivada amplificaría el ruido. Para evitar la derivada se hace el cambio de variable $\tilde{\dot{\textbf{x}_b}}=\dot{\textbf{Z}}+\textbf{K}_e \dot{ x_a}$ y se obtiene:
	
\begin{equation}
\dot{\textbf{Z}}=(\textbf{A}_{bb}-\textbf{K}_e\textbf{A}_{ab})\tilde {\textbf{x}_b}+ \textbf{A}_{ba}x_a+\textbf{B}_bu-\textbf{K}_e(A_{aa}x_a+B_au)
\end{equation}
	
Agrupando términos se tiene:
\begin{equation}
\dot{\textbf{Z}}=(\textbf{A}_{bb}-\textbf{K}_e\textbf{A}_{ab})\textbf{Z}+ 
[(\textbf{A}_{bb}- \textbf{K}_e\textbf{A}_{ab})\textbf{K}_e 
+\textbf{A}_{ba} -\textbf{K}_e Aaa]y  + (\textbf{B}_b-\textbf{K}_e\textbf{B}_a)u 
\end{equation}
	
Para obtener las variables observadas, es decir $\mathbf{\tilde{x}}_b$, se integra $\mathbf{Z}$ y se emplea la ecuación 
\begin{equation}
\tilde{{\textbf{x}_b}}={\textbf{Z}}+\textbf{K}_e { x_a}
\end{equation}




## Ejercicios

**Ejercicio 1**

Diseñe un control por retroalimentación de estados para el siguiente sistema. 
<img src="img/controlt/buck.png" width="400px"> 
Se desea que los polos se ubiquen en s=-3000 $\pm$ j3000. Con E=12V, L=1mH, C=100$\mu$F, R=12.5$\Omega$. Existen en Matlab el comando:

[K]=ppol(A,B,poles) [Scilab], [K]=place(A,B,poles) [Matlab]

para obtener la matriz de ganancias K en función de la ubicación de los polos y comparar con los cálculos realizados en su libreta.


**Ejercicio 2**

Realizar un control por retroalimentacion de estados para que el sistema sea estable. Obtenga sus conclusiones y muestre la simulación.
\begin{eqnarray}
A&=&
\begin{bmatrix}
0 & 1 & 0\\
x_{03}^2/mx_{01}^2 & 0 & -2x_{03}/mx_{01}\\
0 & 0 & -R/L
\end{bmatrix}\\
B&=&\begin{bmatrix}
0\\
0\\
1/L\\
\end{bmatrix}\\
C&=& \begin{bmatrix}
1~ & ~0 & 0\\
\end{bmatrix}
\end{eqnarray}
Con $x_{03}=\sqrt{mgx_{01}}$, R = 0.5$\Omega$, L = 10mH; m = 20mg; g = 9.8m/$s^2$; $x_{01}$ = 0.03 m.

**Ejercicio 3**

Emplee la retroalimentación de estados con el control integral para diseñar un controlador para que los polos se ubiquen en -200, -30+j30, -30-j30 de la siguiente función : 
		
\begin{eqnarray}
A=
\begin{bmatrix}
0 & 1 \\
0 & 25 \\ 
\end{bmatrix} 
B=
\begin{bmatrix}
0  \\
1  \\
\end{bmatrix} \:
C=
\begin{bmatrix}
2500 & 0 
\end{bmatrix} 
\end{eqnarray}
	
**Ejercicio 4**

Diseñe un control por retroalimentación de estados con integrador para que el sistema de levitación magnética para que los polos se ubique en:  -100, -10, -150+j150, -150-j150.

**Ejercicio 5**				

Realice el controlador para el reductor (buck) para que los polos se ubiquen en -3000, -3000+j3000 y -3000-j3000. En PSIM realizar el circuito y el control en simulink.

**Ejercicio 6**

Considere el diseño de un regulador para el sistema representado mediante variables de estado, con:
\begin{eqnarray}
A=
\begin{bmatrix}
0 & 1 \\
20.6 & 0 \\ 
\end{bmatrix} 
\:
B=
\begin{bmatrix}
0 \\
1  \\
\end{bmatrix} \:
C=
\begin{bmatrix}
1 & 0 
\end{bmatrix} 
\end{eqnarray}

Los polos en lazo cerrado del sistema se desean en $s_{12}=-1.8 \pm j1.8$. Suponga un control por realimentación del estado observado\\
\begin{equation}
u=-\textbf{K}\tilde{\textbf{x}}
\end{equation}
con polos del observador en $s_1=-8$ y $s_2=-8$. Encuentre $\textbf{K}$ y $\textbf{K}_e$.

**Ejercicio 7**

Para un motor de CD, con las siguientes parámetros :
$K_m=0.06$, $J_m=80.45e-6$, $L=0.003$, $R=3.13$, $B_m=20e-6$.

*  Obtenga su representación en variables de estado.
*  Diseñe un observador de orden mínimo para observar el par aplicado.
*  Seleccione los valores de la ubicación de los polos del observador y del motor (los valores empleados deben ser únicos, es decir no pueden repetir entre equipos).
